In [5]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import swan_vis as swan


p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [6]:
h5 = '../cerberus_annot.h5'
ab = '../../talon/human_talon_abundance.tsv'
filt_ab = '../cerberus_filtered_abundance.tsv'
obs_col = 'sample'
min_tpm = 1
major_set = 'isos_sample_gene_90.tsv'
swan_file = 'swan.p'
mane_file = o = '/Users/fairliereese/mortazavi_lab/data/rnawg/refs/v40_gene_metadata.tsv'

In [9]:
sg = swan.read(swan_file)
ca = cerberus.read(h5)

Read in graph from swan.p


In [1]:
# plot
# if Mane is principal
# - exp of mane vs. exp of secondary
# if mane != principal
# - exp of mane vs. exp of principal

In [11]:
def uses_principal_feat(x):
    """
    Agg function to determine if individual feats
    (tss, ic, tes) use the principal version of that 
    feature
    """
    return '1' in x.unique()

def uses_principal_iso(x):
    """
    Agg function to determine if the isoform triplet
    is the principal version of that gene
    """
    return '1,1,1' in x.unique()

def count_major_principal_feats(df, **kwargs):
    """
    Count the number of samples that the major isoform
    is the principal isoform and vice versa. Do the same
    for individual features as well.
    
    Parameters:
        sg (swan_vis SwanGraph): SwanGraph with transcript
            abundance data added
            
    Returns:
        df (pandas DataFrame): DF with # samples per gene 
            that use the principal feature as their major
            feature / iso
    """

    # df = get_major_feats(sg, **kwargs)
    # df = major_df.copy(deep=True)

    # drop unnecessary columns
    temp = df.copy(deep=True)
    drop_cols = [c for c in temp.columns if 'pi' in c]
    drop_cols += [c for c in temp.columns if '_id' in c]
    drop_cols += ['tid', 'gid']
    temp.drop(drop_cols, axis=1, inplace=True)

    temp = temp.groupby([obs_col, 'gname',
                        'gid_stable'],
                        observed=True).agg({'tss': uses_principal_feat,
                                            'ic': uses_principal_feat,
                                            'tes': uses_principal_feat,
                                            'triplet': uses_principal_iso}).reset_index()

    # convert from detection bool to int so we can sum up
    # also add the opposiite number
    for feat in ['tss', 'ic', 'tes', 'triplet']:
        col = 'not_{}'.format(feat)
        temp[feat] = temp[feat].astype(int)
        temp[col] = (temp[feat] == False).astype(int)

    # count n samples where major != principal and vice versa
    temp = temp.groupby(['gname', 'gid_stable']).sum().reset_index()

    # total number of samples that this gene is expressed / 
    # has a complete isoform in 
    temp['n_samples'] = temp.triplet+temp.not_triplet

    for feat in ['triplet', 'tss', 'ic', 'tes']:
        col = 'not_{}'.format(feat)
        n_genes = len(temp.gid_stable.unique())
        n_num = len(temp.loc[temp[col] >= 1].index)
        print()
        print('{:.2f}% ({}/{}) genes with >= 1 sample where the major {} is not MANE'.format((n_num/n_genes)*100, n_num, n_genes, feat)) 

    for feat in ['triplet', 'tss', 'ic', 'tes']:
        col = 'perc_{}'.format(feat)
        temp[col] = (temp[feat]/temp.n_samples)*100

    return temp

def get_major_principal_feats(df, **kwargs):
    """
    Determine how many genes have at least one sample where
    the features and full-length isoform is the principal isoform
    
    Parameters:
        sg (swan_vis SwanGraph): SwanGraph with transcript
                    abundance data added   
                    
    Returns:
        df (pandas DataFrame): DF w/ boolean columns for 
            whether or not there's at least one sample
            where the major feat / isoform is the principal one
    """
    # df = get_major_feats(sg, **kwargs)
    # df = major_df.copy(deep=True)

    # add feature numbers
    for feat in ['tss', 'ic', 'tes']:
        id_col = '{}_id'.format(feat)
        df[feat] = df[id_col].str.split('_', expand=True)[1]

    # add triplet
    df['triplet'] = df.tid.str.split('[', expand=True)[1].str.split(']', expand=True)[0]


    # remove unnecessary columns and determine which genes
    # have at least one sample who's major iso uses the principal
    # of each feature
    temp = df.copy(deep=True)
    drop_cols = [c for c in temp.columns if 'pi' in c]
    drop_cols += [c for c in temp.columns if '_id' in c]
    drop_cols += ['tid', 'gid']
    temp.drop(drop_cols, axis=1, inplace=True)

    temp = temp.groupby(['gname',
                         'gid_stable']).agg({'tss': uses_principal_feat,
                                             'ic': uses_principal_feat,
                                             'tes': uses_principal_feat,
                                             'triplet': uses_principal_iso}).reset_index()

    # output
    for feat in ['ic', 'tss', 'tes', 'triplet']:
        n_genes = len(temp.gid_stable.unique().tolist())
        n_major = len(temp.loc[temp[feat] == True].index)
        print(n_genes)
        print(n_major)
        print(feat)
        print('{:.2f}% of genes have >=1 sample where major {} is MANE'.format((n_major/n_genes)*100, feat))
        print()

    return temp

def plot_major_principal_feat_counts(df, opref='figures/', **kwargs):
    """
    Plot a histogram of the number of datasets where the major isoform
    is the principal isoform.
    
    Parameters:
        sg (swan_vis SwanGraph): SwanGraph with transcript
                abundance data added
    
    Returns:
        temp (pandas DataFrame): Output from `count_major_principal_feats`.
    """
    temp = count_major_principal_feats(df, **kwargs)
    mpl.rcParams['font.family'] = 'Arial'
    mpl.rcParams['pdf.fonttype'] = 42
    sns.set_context('paper', font_scale=1.8)
    c_dict, order = get_feat_triplet_colors()  
    for feat in ['triplet', 'tss', 'ic', 'tes']:
        col = 'perc_{}'.format(feat)
        ax = sns.displot(data=temp,
                         x=col,
                         linewidth=0, 
                         color=c_dict[feat],
                         binwidth=5,
                         alpha=1)
        ylabel = '# genes'
        if feat in ['tss', 'ic', 'tes']: 
            xlabel = '% of samples where major {} is from MANE'.format(feat.upper())
        else:
            xlabel = '% of samples where major isoform is MANE'
            
        ax.set(ylabel=ylabel, xlabel=xlabel)

        fname = '{}/MANE_vs_major_{}_hist.pdf'.format(opref, feat)
        plt.savefig(fname, dpi=800, bbox_inches='tight')
        
    return temp


def compute_feat_tpm(adata, obs_col, feat, how, min_tpm=None):
    tpm_df = swan.calc_tpm(adata, obs_col=obs_col, how=how)
    tpm_df = tpm_df.sparse.to_dense()
    tpm_df = tpm_df.T
    tpm_df = tpm_df.melt(var_name=obs_col, value_name='tpm', ignore_index=False)
    tpm_df.reset_index(inplace=True)
    tpm_df.rename({'index': id_col}, axis=1, inplace=True)
    if min_tpm:
        tpm_df = tpm_df.loc[tpm_df.tpm >= min_tpm]
    tpm_df.rename({'tpm': '{}_tpm'.format(feat)}, axis=1, inplace=True)
    return tpm_df

def get_major_feats(sg, obs_col='sample', gene_subset=None, min_tpm=None):
    """
    Determine what the major isoform, tss, tes, ic is in each grouping
    for each gene 
    
    Parameters:
        sg (swan_vis SwanGraph): SwanGraph with transcript
            abundance data added
        obs_col (str): Column in `sg.adata.obs` to group
            datasets by 
        gene_subset (list of str or None): If not None, 
            list of stable gene ids to subset on 
            
    Returns:
        df (pandas DataFrame): DF where each row is the major 
            isoform in each expressed gene / sample combination
    """
    # get the major tss, ic, tes, and isoform from each sample
    adatas = [sg.adata, sg.tss_adata, sg.ic_adata, sg.tes_adata]
    t_dfs = [sg.t_df, sg.tss_adata.var, sg.ic_adata.var, sg.tes_adata.var]
    feats = ['triplet', 'tss', 'ic', 'tes']
    id_cols = {'triplet': 'tid', 'tss': 'tss_id', 'ic': 'ic_id', 'tes': 'tes_id'}
    major_df = pd.DataFrame()

    for feat, adata, t_df in zip(feats, adatas, t_dfs):

        # get pi value for each feature in the specified gb category
        df, _ = swan.calc_pi(adata, t_df, obs_col=obs_col)
        df = df.sparse.to_dense()
        df = df.transpose()

        # merge with gene info
        id_col = id_cols[feat]
        t_df = t_df.copy(deep=True)
        if feat == 'triplet':
            drop = True
        else:
            drop = False
        t_df.reset_index(drop=drop,inplace=True)
        t_df = t_df[[id_col, 'gid']]
        df = df.merge(t_df, how='inner', on=id_col)
        df.set_index([id_col, 'gid'], inplace=True)

        # pivot and coerce into lengthwise format
        df = df.melt(ignore_index=False, value_name='pi', var_name=obs_col)
        df = df.dropna(subset=['pi'])
        df.reset_index(inplace=True)

        # enforce min tpm for isoforms
        if min_tpm and feat == 'triplet':
            tpm_df = compute_feat_tpm(adata, obs_col, feat, how='max', min_tpm=min_tpm)
            df = df.merge(tpm_df, how='inner', on=[id_col, 'sample'])
            df.drop('{}_tpm'.format(feat), axis=1, inplace=True)

        # merge in avg. tpm for each feat and enforce min
        tpm_df = compute_feat_tpm(adata, obs_col, feat, how='mean')
        df = df.merge(tpm_df, how='left', on=[id_col, 'sample'])

        # remove unexpressed isoforms
        df = df.loc[df.pi > 0]

        # limit to detected genes
        df['gid_stable'] = cerberus.get_stable_gid(df, 'gid')
        if gene_subset:
            df = df.loc[df.gid_stable.isin(gene_subset)]

        # sort by gene, sample, and pi value
        # dedupe across the gene and sample cols; take the top-expressed isoform
        df = df.sort_values(by=['gid', obs_col, 'pi', '{}_tpm'.format(feat)],
                            ascending=[False, False, False, False])

        df = df.drop_duplicates(subset=['gid', obs_col], keep='first')

        # rename some columns
        if feat == 'triplet': 
            df.rename({'pi': 'tid_pi'}, axis=1, inplace=True)
        else:
            pi_col = '{}_pi'.format(feat)
            df.rename({'pi': pi_col}, axis=1, inplace=True)

        # first entry
        if major_df.empty:
            major_df = df.copy(deep=True)
        else:
            major_df = major_df.merge(df, how='outer', on=['gid', 'sample', 'gid_stable'])        

    # add gene name
    g_df = sg.t_df[['gid', 'gname']].reset_index(drop=True).drop_duplicates()
    major_df = major_df.merge(g_df, how='left', on='gid')
    
    # add feature numbers
    for feat in ['tss', 'ic', 'tes']:
        id_col = '{}_id'.format(feat)
        major_df[feat] = major_df[id_col].str.split('_', expand=True)[1]

    # add triplet
    major_df['triplet'] = major_df.tid.str.split('[', expand=True)[1].str.split(']', expand=True)[0]

    return major_df


In [12]:
def plot_mane_v_princ_tpm(temp, feat):
    temp['log_princ'] = np.log2(temp['{}_tpm_principal'.format(feat)]+1)
    temp['log_mane'] = np.log2(temp['{}_tpm_mane'.format(feat)]+1)

    lim_max = max(temp.log_princ.max(), temp.log_mane.max())
    xlim = ylim = (-2, lim_max)

    c_dict, order = get_feat_triplet_colors()  
    color = c_dict[feat]

    ax = sns.jointplot(data=temp, x='log_mane', y='log_princ', color=color, 
                       xlim=xlim, ylim=ylim)

    if feat == 'triplet':
        label = 'isoform'
    else:
        label = feat.upper()
    ylabel = 'log2(Principal {} TPM+1)'.format(label)
    xlabel = 'log2(MANE {} TPM+1)'.format(label)

    tick_range = range(0, int(lim_max+1), 5)
    ax.ax_joint.set_xticks(tick_range)
    ax.ax_joint.set_yticks(tick_range)
    ax.ax_joint.set(ylabel=ylabel, xlabel=xlabel)
    
    fname = 'figures/mane_vs_principal_{}_tpm.pdf'.format(feat)
    plt.savefig(fname, dpi=800, bbox_inches='tight')

In [ ]:
adatas = [sg.adata, sg.tss_adata, sg.ic_adata, sg.tes_adata]
t_dfs = [sg.t_df, sg.tss_adata.var, sg.ic_adata.var, sg.tes_adata.var]
feats = ['triplet', 'tss', 'ic', 'tes']
id_cols = {'triplet': 'tid', 'tss': 'tss_id', 'ic': 'ic_id', 'tes': 'tes_id'}
id_cols_2 = {'triplet': 'transcript_id', 'tss': 'tss_id', 'ic': 'ic_id', 'tes': 'tes_id'}
how = 'mean'

for feat, adata, t_df in zip(feats, adatas, t_dfs):
    print(feat)
    id_col = id_cols[feat]
    id_col_2 = id_cols_2[feat]
    tpm_df = compute_feat_tpm(adata, obs_col, feat, how, min_tpm=None)
    pdb.set_trace()
    
    # get mane tids -- look at v40 metadata and cross ref. w/ cerberus
    # also grab their TPMs
    meta_df = pd.read_csv('../../../refs/v40_transcript_metadata.tsv', sep='\t')
    mane_tids = meta_df.loc[meta_df.MANE_Select].tid.tolist()
    mane_df = ca.t_map.copy(deep=True)
    mane_df = mane_df.loc[mane_df.source=='v40']
    mane_df = mane_df.loc[mane_df.original_transcript_id.isin(mane_tids)]
    mane_feats = mane_df[id_col_2].unique().tolist()
    mane_feats[:5]

    # limit tpm_df to just the mane features
    print(len(tpm_df.index))
    tpm_df = tpm_df.loc[tpm_df[id_col].isin(mane_feats)]
    print(len(tpm_df.index))

    # add the gene id 
    if feat == 'triplet':
        pat = '['
    else:
        pat = '_'
    tpm_df['gid'] = tpm_df[id_col].str.split(pat, expand=True)[0]
    tpm_df.head()

    # merge in TPM of MANE features
    df = df.merge(tpm_df, how='left', on=['gid', obs_col], suffixes=('_principal', '_mane'))
    # fill nans for tpm w/ 0
    df['{}_tpm_mane'.format(feat)].fillna(0, inplace=True)

    prin_mane_col = '{}_principal_is_mane'.format(feat)
    df[prin_mane_col] = df[id_col+'_principal']==df[id_col+'_mane']

    # when is the mane isoform not the principal isoform, but mane is still expressed?
    temp = df.loc[df[prin_mane_col] == False]
    n = len(temp.index)
    temp = temp.loc[temp['{}_tpm_mane'.format(feat)]>0]
    n_num = len(temp.index)
    print('{:.2f}% ({}/{}) of gene / sample combos have MANE expression where principal {} is not MANE '.format((n_num/n)*100, n_num, n, feat))
    plot_mane_v_princ_tpm(temp, feat)
    
    # when is the mane isoform not the principal isoform, and mane is NOT expressed?
    temp = df.loc[df[prin_mane_col] == False]
    n = len(temp.index)
    temp = temp.loc[temp['{}_tpm_mane'.format(feat)]==0]
    n_num = len(temp.index)
    print('{:.2f}% ({}/{}) of gene / sample combos have NO MANE expression where principal {} is not MANE '.format((n_num/n)*100, n_num, n, feat))

    print()

triplet
> <ipython-input-13-1737c85d370d>(17)<module>()
     15     # get mane tids -- look at v40 metadata and cross ref. w/ cerberus
     16     # also grab their TPMs
---> 17     meta_df = pd.read_csv('../../../refs/v40_transcript_metadata.tsv', sep='\t')
     18     mane_tids = meta_df.loc[meta_df.MANE_Select].tid.tolist()
     19     mane_df = ca.t_map.copy(deep=True)



ipdb>  tpm_df.head()


                      tid sample  triplet_tpm
0  ENSG00000227232[1,2,1]  caco2     1.435214
1  ENSG00000227232[1,3,1]  caco2     1.799986
2  ENSG00000227232[1,4,1]  caco2     0.432952
3  ENSG00000227232[1,5,1]  caco2     0.501131
4  ENSG00000227232[2,2,1]  caco2     0.000000
